In [3]:
from neo4j import GraphDatabase
import pandas as pd

# Dataset

## ESCO Skill Dataset

In [4]:
skills = pd.read_csv('../data/all_skills.csv')[['concept_uri','preferred_label','description']]

## DIN PAS 1045 Course Description Datase

In [5]:
courses = pd.read_csv('../data/all_courses.csv')[['course_id','course_name','course_description']]

# Neo4J CRUD

In [16]:
class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()
        
    def create_skill(self, concept_uri, preferred_label, description):
        with self.driver.session() as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.write_transaction(self._create_and_return_skill, 
                                               concept_uri, preferred_label, description)
            
    @staticmethod
    def _create_and_return_skill(tx, concept_uri, preferred_label, description):
        query = (
            "CREATE (skill:Skill {concept_uri: $concept_uri, \
            preferred_label: $preferred_label, description: $description})"
            "RETURN skill"
        )

        results = tx.run(query, concept_uri=concept_uri, 
                             preferred_label=preferred_label, 
                             description=description)
    
    def create_skills(self, csv_path):
        with self.driver.session() as session:
            result = session.write_transaction(self._create_skills, csv_path)
            
    @staticmethod
    def _create_skills(tx, csv_path):
        assert "http" in csv_path, "Loading csv files from local is \
        not possible due to security reasons, please use online files." 
        query = (
            "LOAD CSV WITH HEADERS FROM $csv_path AS skill \
            CREATE (:Skill {concept_uri: skill.concept_uri, \
            preferred_label: skill.preferred_label, description: skill.description})"
        )
        results = tx.run(query, csv_path=csv_path)
        
    def create_course(self, course_name, course_id, course_description):
        with self.driver.session() as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.write_transaction(self._create_and_return_course, 
                                               course_name, course_id, course_description)
            
    @staticmethod
    def _create_and_return_course(tx, course_name, course_id, course_description):
        
        query = (
            "CREATE (course:Course {course_id: $course_id, course_name: \
            $course_name, course_description: $course_description})"
            "RETURN course"
        )

        results = tx.run(query, course_id=course_id, 
                             course_name=course_name, 
                             course_description=course_description)
        
    def create_courses(self, csv_path):
        with self.driver.session() as session:
            result = session.write_transaction(self._create_courses, csv_path)
            
    @staticmethod
    def _create_courses(tx, csv_path):
        assert "http" in csv_path, "Loading csv files from local is \
        not possible due to security reasons, please use online files." 
        query = (
            "LOAD CSV WITH HEADERS FROM $csv_path AS course \
            CREATE (:Course {course_id: course.course_id, \
            course_name: course.course_name, course_description: course.course_description})"
        )
        results = tx.run(query, csv_path=csv_path)
    
    def create_relation(self, course_id, concept_uri):
        with self.driver.session() as session:
            result = session.write_transaction(self._create_relation, course_id, concept_uri)
    
    @staticmethod
    def _create_relation(tx, course_id, concept_uri):
        query = (
            "MATCH (course:Course), (skill:Skill) \
            WHERE course.course_id = $course_id AND skill.concept_uri = $concept_uri \
            CREATE (course)-[provide_skill:RELTYPE]->(skill) \
            RETURN type(r)"
        )
        results = tx.run(query, course_id=course_id, concept_uri=concept_uri)

In [17]:
if __name__ == "__main__":
    # Aura queries use an encrypted connection using the "neo4j+s" URI scheme
    uri = "neo4j+s://fbb398e7.databases.neo4j.io"
    user = "neo4j"
    password = "3bO4Zwc9eYOyZBQ_CHE49lqd7NKeLSySJAq4flxUtAA"
    app = App(uri, user, password)
    app.create_skills("https://raw.githubusercontent.com/ingastrelnikova/awt-pj-ss22-learn-tech-2/main/data/all_skills.csv")
    app.create_courses("https://raw.githubusercontent.com/ingastrelnikova/awt-pj-ss22-learn-tech-2/main/data/all_courses.csv")
    app.create_relation("99BEDCEB-4FF3-3F8F-88B7-B30E50638F01","http://data.europa.eu/esco/skill/08fe5910-8c4c-4679-a988-eb2ded3c4909")
    app.close()

DatabaseError: {code: Neo.DatabaseError.Statement.ExecutionFailed} {message: At https://raw.githubusercontent.com/ingastrelnikova/awt-pj-ss22-learn-tech-2/main/data/all_courses.csv @ position 9893221 -  there's a field starting with a quote and whereas it ends that quote there seems to be characters in that field after that ending quote. That isn't supported. This is what I read: '- die individuelle Selbstanalyse: Motive, Stärken, Defizite und Zielvorstellungen
- Persönlichkeit und Motivation in Organisationen: Wirkung der drei Motive:
  1. Leistung
  2. Funktionsorientierung
  3. Macht
- Leitung und Leistung: die eigene Leistung richtig einschätzen
- Durchsetzung im beruflichen Umfeld
- Selbstsicherheit erlernen durch die Techniken ""FF'}

CREATE (ee:Person { name: "Emil", from: "Sweden", klout: 99 })

MATCH (ee:Person) WHERE ee.name = "Emil" RETURN ee;

CREATE (js:Person { name: "Johan", from: "Sweden", learn: "surfing" }),
(ir:Person { name: "Ian", from: "England", title: "author" }),
(rvb:Person { name: "Rik", from: "Belgium", pet: "Orval" }),
(ally:Person { name: "Allison", from: "California", hobby: "surfing" }),
(ee)-[:KNOWS {since: 2001}]->(js),(ee)-[:KNOWS {rating: 5}]->(ir),
(js)-[:KNOWS]->(ir),(js)-[:KNOWS]->(rvb),
(ir)-[:KNOWS]->(js),(ir)-[:KNOWS]->(ally),
(rvb)-[:KNOWS]->(ally)

MATCH
  (a:Person),
  (b:Movie)
WHERE a.name = 'Allison' AND b.title = 'The Matrix Revolutions'
CREATE (a)-[r:ACTED_IN]->(b)
RETURN type(r)

MATCH (n) DETACH DELETE n


MATCH (n) RETURN n